In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:

spark = SparkSession.builder.getOrCreate()

22/09/12 13:46:38 WARN Utils: Your hostname, Jarads-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.27 instead (on interface en0)
22/09/12 13:46:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 13:46:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/12 13:46:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/12 13:46:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/12 13:46:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [5]:

# Read the case, department, and source data into their own spark dataframes.
source = spark.read.csv('source.csv', sep= ',', header=True, inferSchema=True)
case = spark.read.csv('case.csv', sep= ',', header=True, inferSchema=True)
dept = spark.read.csv('dept.csv', sep= ',', header=True, inferSchema=True)

In [6]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [7]:
case.show(5)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

In [8]:
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



Let's see how writing to the local disk works in spark:

Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [9]:
# write to json
source.write.json("sources_json", mode="overwrite")

In [10]:
# write to csv
source.write.csv("sources_csv", mode="overwrite")

nspect your folder structure. What do you notice?

In [11]:
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [12]:
# Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [13]:
# to_timestamp, fmt

fmt = "M/d/yy H:mm"

case = case.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))\
.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))\
.withColumn('SLA_due_date', to_timestamp('SLA_due_date', fmt))

In [14]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [15]:
case = case.withColumn('case_closed', expr('case_closed == "YES"'))\
.withColumn('case_late', expr('case_late == "YES"'))

Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [16]:

source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [17]:
(
    case.select(datediff(current_timestamp(), 'SLA_due_date')
    .alias('days_past_due'))
    .where(case.case_closed == False)
    .sort(col('days_past_due').desc())
    .show(5)
    
)

+-------------+
|days_past_due|
+-------------+
|         2064|
|         2064|
|         2064|
|         2063|
|         2061|
+-------------+
only showing top 5 rows




2. How many Stray Animal cases are there?

In [18]:
case.filter(case.service_request_type == 'Stray Animal').count()

26760

3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [19]:
(
    case.select('service_request_type')
    .where(case.dept_division == 'Field Operations')
    .where(case.service_request_type != 'Officer Standby').count()
)

113902

4. Convert the council_district column to a string column.

In [21]:
case = case.withColumn("council_district", col("council_district").cast("string"))

case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('SLA_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string')]

5. Extract the year from the case_closed_date column.

In [22]:

case = case.withColumn('year', year("case_closed_date"))

case.select(case.year).show(5)

+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 5 rows



22/09/12 16:46:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 305371 ms exceeds timeout 120000 ms
22/09/12 16:46:49 WARN SparkContext: Killing executors is not supported by current scheduler.
